# Walk Planner for Rehabilitation

In mid-2021, I suffered an adverse reaction to some medication that left me very weak and unable to perform heavy physical exercise. As part of my plan for rehabilitation, I decided to set a goal of walking a certain distance every day, at first just 2km. However, I quickly got bored of walking the same route so decided to develop this planner to find interesting destinations to walk to so I could get my distance in!

In this notebook, all you have to do is enter your location, how far you want to walk (in total) and it will pull recommendations of interesting destinations in the relevant radius from FourSquare, then cross-reference them with Google Maps and provide you with a list of the top 25 things you can walk to to help keep your walk fresh.

In [1]:
import requests 
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np 
import random
import math
#!pip install geopy
from geopy.geocoders import Nominatim
#!pip install folium==0.5.0
import folium
#import config

print('Libraries imported.')

Libraries imported.


## Enter your details
First, we need to know where your starting point is, how far you want to walk in total (kilometers, integer), and what sort of attraction you want to walk to - it can be restaurants, bars, parks... anything you could find on the FourSquare app!

In [2]:
address = str(input("Please enter your address - number, street name and postcode/zip code should suffice. \n"))
print(f'Please run the next cell to check the address is correct!')

Please enter your address - number, street name and postcode/zip code should suffice. 
colon 603 oaxaca 68000
Please run the next cell to check the address is correct!


In [3]:
geolocator = Nominatim(user_agent='facebook')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(f"{address}'s latitude is {latitude} and its longitude is {longitude}")
print("The full address is:")
print(location)
print('If this is not correct, please retry entering your address above.')

colon 603 oaxaca 68000's latitude is 17.058713 and its longitude is -96.7232573
The full address is:
Calle de Cristóbal Colón, Oaxaca, Oaxaca de Juárez, Oaxaca, 68090, México
If this is not correct, please retry entering your address above.


In [4]:
distance = int(input("Please enter how many kilometers you want to walk in total: \n"))
print(f"Let's find you interesting things to walk to for a total of {distance}km.")

Please enter how many kilometers you want to walk in total: 
5
Let's find you interesting things to walk to for a total of 5km.


In [5]:
search_string = str(input("Please enter what you would like to search for, e.g., 'Restaurant' or 'Art Gallery', or leave blank to find every interesting location: \n")) or ''
print(f"Let's find you locations related to {search_string}.")

Please enter what you would like to search for, e.g., 'Restaurant' or 'Art Gallery', or leave blank to find every interesting location: 
restaurant
Let's find you locations related to restaurant.


## Set up FourSquare API
Now we have the details, the following code sets up the requests we will pull from the foursquare API

In [6]:
# Foursquare Search Settings
search_query = search_string.replace(' ','+')
search_radius = 250 # meters
exclude_all_chains = "false"
open_now = "false"
sort = 'rating'
limit = 25 #50 is the maximum


In [7]:
# Foursquare Credentials
url = f"https://api.foursquare.com/v3/places/search?query={search_query}&ll={latitude},{longitude}&radius={search_radius}&exclude_all_chains={exclude_all_chains}&open_now={open_now}&sort={sort}&limit={limit}".format(search_query,latitude,longitude,search_radius,exclude_all_chains,open_now,sort)

headers = {
    "Accept": "application/json",
    "Authorization": secrets.API_KEY
}

response = requests.request("GET", url, headers=headers)

#print(response.text)

### Set up the location search
The below code identifies 12 points around your starting location, at a radius defined by the distance you want to walk. Radius will be calculated using Pythagoras theorem, assuming that the waking distance includes roughly equal parts north/south and east/west

In [8]:
# inputs
circle_radius = math.sqrt(((1000*distance)/4)**2+((1000*distance)/4)**2)

# parameters
N = 12 # number of discrete sample points to be generated along the circle

# generate points
circlePoints = []
for k in range(N):
    # compute
    angle = math.pi*2*k/N
    dx = circle_radius*math.cos(angle)
    dy = circle_radius*math.sin(angle)
    point = {}
    point['lat']=latitude + (180/math.pi)*(dy/6378137)
    point['lon']=longitude + (180/math.pi)*(dx/6378137)/math.cos(latitude*math.pi/180)
    # add to list
    circlePoints.append(point)


circle_points_df = pd.DataFrame(circlePoints)
print(circle_points_df.head())

         lat        lon
0  17.058713 -96.706646
1  17.066653 -96.708872
2  17.072466 -96.714952
3  17.074593 -96.723257
4  17.072466 -96.731563


### Search FourSquare for Recommendations
Now we have the list of points around the city to search, I'll run a for-loop to build up a dataframe of all relevant locations at the correct distance.

In [10]:
# Get list of JSON results from Foursquare for all points in circle
json_results = []
for lat, lng in zip(circle_points_df['lat'], circle_points_df['lon']):
    latitude1 = lat
    longitude1 = lng
    url = f"https://api.foursquare.com/v3/places/search?query={search_query}&ll={latitude1},{longitude1}&radius={search_radius}&exclude_all_chains={exclude_all_chains}&open_now={open_now}&sort={sort}&limit={limit}"
    response = requests.request("GET", url, headers=headers)
    json_results.append(response.json())

In [11]:
# Concatanate results for all boroughs to one dataframe 
location_data = []

for i in range(len(json_results)):
    for j in range(len(json_results[i]['results'])):
        location = json_results[i]['results'][j]
        location_as_df = pd.json_normalize(location)
        location_data.append(location_as_df)
        
all_locations_df = pd.concat(location_data)
all_locations_df.reset_index().head()

,index,fsq_id,categories,chains,distance,name,timezone,geocodes.main.latitude,geocodes.main.longitude,location.country,location.cross_street,location.formatted_address,location.locality,location.postcode,location.region,location.address,geocodes.roof.latitude,geocodes.roof.longitude,location.neighborhood
0,0,4ff0ce0be4b00b9a295b2a9e,"[{'id': 13338, 'name': 'Seafood Restaurant', '...",[],234,El Pargo,America/Mexico_City,17.068498,-96.710061,MX,,"Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,NaN,NaN,NaN,NaN
1,0,51375e66e4b027d72a242ec1,"[{'id': 13306, 'name': 'Taco Restaurant', 'ico...",[],62,Doña Olga,America/Mexico_City,17.066162,-96.708502,MX,,"Santa Lucía, Oaxaca",Santa Lucía,,Oaxaca,NaN,NaN,NaN,NaN
2,0,527b0d79498ef69dfe2228d5,"[{'id': 13031, 'name': 'Burger Joint', 'icon':...",[],118,Krusty Burger,America/Mexico_City,17.066883,-96.709958,MX,,"Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,NaN,NaN,NaN,NaN
3,0,4f3746e5e4b0beedd1e28e3d,"[{'id': 13303, 'name': 'Mexican Restaurant', '...",[],201,Tlayudas de Lucy,America/Mexico_City,17.064734,-96.709205,MX,,"Calle Oriente 8, Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,Calle Oriente 8,NaN,NaN,NaN
4,0,569b0b6e498eb2bf9047147d,"[{'id': 13306, 'name': 'Taco Restaurant', 'ico...",[],231,Tacos Puebla,America/Mexico_City,17.065020,-96.710232,MX,,"Santa Lucía, Oaxaca",Santa Lucía,,Oaxaca,NaN,NaN,NaN,NaN


In [12]:
### EXPORT CSV TO BACKUP FOR TEST
#all_locations_df.to_csv('Oaxaca-locations-DF.csv')
### IMPORT CSV IF NEEDED
#all_locations_df = pd.read_csv('Oaxaca-locations-DF.csv')

### Clean the data

In [13]:
# Remove Duplicates
all_locations_df.drop_duplicates(subset=['name'], inplace = True)

# Clean Categories
all_locations_df['categories'] = all_locations_df['categories'].apply(lambda x:x[0].get('name'))#all_locations_df.head()

# Rename Columns
rename_columns = {'geocodes.main.latitude':'latitude','geocodes.main.longitude':'longitude','location.address':'address','location.formatted_address':'formatted_address'}
all_locations_df = all_locations_df.rename(rename_columns, axis=1)

all_locations_df.reset_index().head()

,index,fsq_id,categories,chains,distance,name,timezone,latitude,longitude,location.country,location.cross_street,formatted_address,location.locality,location.postcode,location.region,address,geocodes.roof.latitude,geocodes.roof.longitude,location.neighborhood
0,0,4ff0ce0be4b00b9a295b2a9e,Seafood Restaurant,[],234,El Pargo,America/Mexico_City,17.068498,-96.710061,MX,,"Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,NaN,NaN,NaN,NaN
1,0,51375e66e4b027d72a242ec1,Taco Restaurant,[],62,Doña Olga,America/Mexico_City,17.066162,-96.708502,MX,,"Santa Lucía, Oaxaca",Santa Lucía,,Oaxaca,NaN,NaN,NaN,NaN
2,0,527b0d79498ef69dfe2228d5,Burger Joint,[],118,Krusty Burger,America/Mexico_City,17.066883,-96.709958,MX,,"Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,NaN,NaN,NaN,NaN
3,0,4f3746e5e4b0beedd1e28e3d,Mexican Restaurant,[],201,Tlayudas de Lucy,America/Mexico_City,17.064734,-96.709205,MX,,"Calle Oriente 8, Oaxaca de Juárez, Oaxaca",Oaxaca,,Oaxaca,Calle Oriente 8,NaN,NaN,NaN
4,0,569b0b6e498eb2bf9047147d,Taco Restaurant,[],231,Tacos Puebla,America/Mexico_City,17.065020,-96.710232,MX,,"Santa Lucía, Oaxaca",Santa Lucía,,Oaxaca,NaN,NaN,NaN,NaN


### Plot Suggestions to a Map
Now we have a list of suggestions around your starting location, we can plot them to a map using Folium.

In [16]:
location_suggestions_map = folium.Map(location=[latitude, longitude], zoom_start=14)

# Add starting location to the map
folium.CircleMarker(location=[latitude, longitude], radius=6, color='red', fill=False).add_to(location_suggestions_map) 

# Add location markers to map
for lat, lng, label, address in zip(
    all_locations_df['latitude'], 
    all_locations_df['longitude'], 
    all_locations_df['name'],
    all_locations_df['formatted_address']):
    label = folium.Popup(f'{label} || {address}', parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(location_suggestions_map)  
    
location_suggestions_map

## Congratulations!
Now you have a map of interesting locations (blue circles) that you can walk to from your starting point (red circle) and back, that will that enable you to walk your target distance every day. I hope this little app helps you in your recovery journey!